In [1]:
###################################################
### Publication or Commercial usage not allowed ###
###          v.rafanavicius@gmail.com           ###
###  If you are writing publication using this  ###
###             code please contact             ###
###################################################
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import folium
import csv
import pandas as pd
import simplekml
import math
from noaa_sdk import noaa
import requests
from datetime import datetime, timedelta
from netCDF4 import num2date
from siphon.catalog import TDSCatalog
from metpy.units import units
import metpy.calc as mc
import geopy
from geopy import Point
from geopy import distance
#TODO: Document the code
#TODO: Speed-up by saving intermediate results
#TODO: Baloon landing site prediction
#TODO: Implement static class instead of inheritance
#TODO: Launch site prediction from landing site (reverse the wind and altitude path)
#TODO: Test with real data
#TODO: Visualization directly in the map + Graphs
#TODO: Write README

In [2]:
class Atmosphere: 
    def pres2alt(self,pressure):
        '''
        Determine altitude from site pressure.

        Parameters
        ----------
        pressure : numeric
            Atmospheric pressure (Pascals)

        Returns
        -------
        altitude : numeric
            Altitude in meters above sea level

        Notes
        ------
        The following assumptions are made

        ============================   ================
        Parameter                      Value
        ============================   ================
        Base pressure                  101325 Pa
        Temperature at zero altitude   288.15 K
        Gravitational acceleration     9.80665 m/s^2
        Lapse rate                     -6.5E-3 K/m
        Gas constant for air           287.053 J/(kgK)
        Relative Humidity              0%
        ============================   ================

        References
        -----------
        .. [1] "A Quick Derivation relating altitude to air pressure" from
           Portland State Aerospace Society, Version 1.03, 12/22/2004.
        '''

        alt = 44331.5 - 4946.62 * pressure ** (0.190263)

        return alt



    def alt2pres(self,altitude):
        '''
        Determine site pressure from altitude.

        Parameters
        ----------
        altitude : numeric
            Altitude in meters above sea level

        Returns
        -------
        pressure : numeric
            Atmospheric pressure (Pascals)

        Notes
        ------
        The following assumptions are made

        ============================   ================
        Parameter                      Value
        ============================   ================
        Base pressure                  101325 Pa
        Temperature at zero altitude   288.15 K
        Gravitational acceleration     9.80665 m/s^2
        Lapse rate                     -6.5E-3 K/m
        Gas constant for air           287.053 J/(kgK)
        Relative Humidity              0%
        ============================   ================

        References
        -----------
        .. [1] "A Quick Derivation relating altitude to air pressure" from
           Portland State Aerospace Society, Version 1.03, 12/22/2004.
        '''

        press = 100 * ((44331.514 - altitude) / 11880.516) ** (1 / 0.1902632)

        return press

class Wind: 
    def __init__(self): 
        self.position = np.array([23,54], dtype=np.float32) 
        self.speed = 0 
        self.direction = 0 
        self.pressure = 100000 * units('Pa') 
        self.names = ['u_wind, m/s', 'v_wind, m/s', 'pressure, Pa'] 
       
  # Copyright (c) 2013-2015 Siphon Contributors. 
  # Distributed under the terms of the BSD 3-Clause License. 
    def get_wind_nccs(self):
        '''
        Get wind components from nccs at different altitudes.

        Parameters
        ----------

        Returns
        -------
        u_wind, v_wind, pressure : netCDF4.Variable
        '''
        position = self.position 
        ########################################### 
        # First we construct a TDSCatalog instance pointing to our dataset of interest, in 
        # this case TDS' "Best" virtual dataset for the GFS global 0.25 degree collection of 
        # GRIB files. We see this catalog contains a single dataset. 

        #definetly needs some error management!!! 
        best_gfs = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/' 
                    'Global_0p25deg/catalog.xml?dataset=grib/NCEP/GFS/Global_0p25deg/Best') 
        ########################################### 
        # We pull out this dataset and get the NCSS access point 
        best_ds = best_gfs.datasets[0] 
        ncss = best_ds.subset() 
        ########################################### 
        # We can then use the `ncss` object to create a new query object, which 
        # facilitates asking for data from the server. 
        query = ncss.query() 
        ########################################### 
        # We construct a query asking for data
        now = datetime.utcnow() 
        query.lonlat_point(position[0], position[1]).time(datetime.utcnow()) 
        ########################################### 
        # We now request data from the server using this query. The `NCSS` class handles parsing
        query.variables('u-component_of_wind_isobaric', 'v-component_of_wind_isobaric') 
        query.accept('netcdf') 
        data = ncss.get_data(query) 
        query = ncss.query() 
        u_wind = data.variables['u-component_of_wind_isobaric'] 
        v_wind = data.variables['v-component_of_wind_isobaric'] 
        pressure = data.variables['isobaric'] 
        #       TODO: SAVE TO DF AND LOAD UNTIL 0p25deg CHANGE
        return u_wind, v_wind, pressure 
    
    def get_wind_data(self, array): 
        nparray = np.array(0) 
        df = pd.DataFrame() 
        for idx in array: 
            nparray = idx.flatten() 
        return nparray.astype(float) 

    def get_wind_df(self): 
        parameters = self.get_wind_nccs() 
        names = ['u_wind, m/s', 'v_wind, m/s', 'pressure, Pa'] 
        df = pd.DataFrame() 
        for name, parameter in zip(names, parameters): 
            df[name] = self.get_wind_data(parameter) 
        return df  
   
    def get_wind_full_df(self): 
        df = self.get_wind_df() 
        directions = [] 
        speeds = [] 
        for ix in range(len(df.index)): 
            u_wind2 = df['u_wind, m/s'].iloc[ix]*units('meter')/units('second') 
            v_wind2 = df['v_wind, m/s'].iloc[ix]*units('meter')/units('second') 
            direction = mc.wind_direction(u_wind2,v_wind2, convention='to') 
            speed = mc.wind_speed(u_wind2,v_wind2) 
            directions.append(direction) 
            speeds.append(speed) 
        df['direction'] = directions 
        df['speed'] = speeds 
        return df 
   
    def save_df(self): 
        df = self.get_wind_full_df() 
        df.to_pickle("weather.pkl") 
       
    def load_df(self): 
        return pd.read_pickle("weather.pkl") 

    def get_wind_interpolate(self, name, load_df=False): 
        value = self.pressure 
        if load_df==False: 
            df = self.get_wind_full_df() 
        else: 
            df = self.load_df() 
      #interpolation function given any pressure value return interpolated wind speed or direction 
        exactmatch=df[df['pressure, Pa']==value] 
        if value > 100000.0: 
            return df[df['pressure, Pa']==100000][name].values.squeeze() 
        elif (exactmatch.empty == False): 
            return exactmatch[name].values.squeeze() 
        else: 
            df1 = df[df['pressure, Pa']<value] 
            df2 = df[df['pressure, Pa']>value] 
            df_min = df1[df1['pressure, Pa'] == df1['pressure, Pa'].max()] 
            df_max = df2[df2['pressure, Pa'] == df2['pressure, Pa'].min()] 
            x_min = df_min['pressure, Pa'].values 
            x_max = df_max['pressure, Pa'].values 
            y_min = (df_min[name]).values 
            y_max = (df_max[name]).values 
            dy = y_max - y_min 
            dx = x_max - x_min 
            A = y_min - (dy/dx) * x_min 
            B = dy/dx 
            y_int = A + B * value 
            return y_int.squeeze() 
       
    def get_wind_direction(self): 
        return self.get_wind_interpolate('direction') 
   
    def get_wind_speed(self): 
        return self.get_wind_interpolate('speed') 
   
     
class Baloon(Wind): 
    def __init__(self, time_step = 600): 
        self.id = 0 
        self.position = np.array([23,54], dtype=np.float32) 
        self.speed = 0  * units('meters')/units('second') 
        self.ascent_rate = 5 * units('meters')/units('second') # m/s 
        self.heading = 0 * units('degree') 
        self.altitude = 0 * units('meter') 
        self.prev_a = 0   
        self.time_step = units.Quantity(time_step, 'second') 
   
    def get_coord_float(self, lat_lon): 
        return [float(idx) for idx in lat_lon.split(',')] 
       
  # Physics function here 
    def get_baloon_coords(self): 
        wind_position = self.position 
        wind_speed = wind.get_wind_speed() 
        direction = self.get_baloon_direction() 
        horizontal_shift = self.get_baloon_dist() 
        direction = np.rad2deg(direction.astype(float)) 
        lat_lon = geopy.distance.geodesic(kilometers=horizontal_shift.magnitude).destination(Point(self.position[1],self.position[0
        ]), direction).format_decimal()
#       TODO: SAVE TO LOG FILE FUNCTION
#         print('dir: ',direction) 
#         print('wind spd: ',wind_speed) 
#         print('h shift: ',horizontal_shift) 
#         print('h shift: ',self.position) 
#         print(lat_lon) 
        x = self.get_coord_float(lat_lon) 
        return x 

  #Calculates horizontal distance of the baloon in one time step 
    def get_baloon_dist(self): 
        wind.position = self.position 
        wind_speed = wind.get_wind_speed() 
        return (wind_speed*units('meter')/units('second')*self.time_step).to('kilometer').astype(float) 
   
    def get_baloon_direction(self): 
        wind.position = self.position 
        return wind.get_wind_direction() 

    def get_baloon_elevation(self): 
        return self.ascent_rate*units('meter')/units('second')*self.time_step 

atm = Atmosphere() 
wind = Wind()     
ball = Baloon() 

In [3]:
#TODO: wrap in some function

#setup parameters
burst_altitude = 20000*units('meter') # meters 
ball.time_step = 500*units('second') # seconds 
ball.position = [22.66829, 55.47072] # initial position

#calculate elevation per one time_step 
elevation = ball.time_step*ball.ascent_rate 
altitude = 0*units('meter') 
lat =[ball.position[1]] 
lon = [ball.position[0]] 
name = [0] 
print(altitude) 

#TODO: last position should be calculated at the burst_altitude not at the last step !!!!

while(altitude <= burst_altitude):     
    wind.pressure = atm.alt2pres(altitude.magnitude) 
    coords = ball.get_baloon_coords() 
    ball.position = np.flip(coords) 
    lat = np.append(lat,coords[0]) 
    lon = np.append(lon,coords[1]) 
    elevation = ball.time_step*ball.ascent_rate 
    altitude += elevation 
    name = np.append(name, altitude.magnitude) 
    print(altitude) 

0 meter
2500.0 meter
5000.0 meter
7500.0 meter
10000.0 meter
12500.0 meter
15000.0 meter
17500.0 meter
20000.0 meter
22500.0 meter


In [5]:
data = {'Name': name.astype('str'),
        'Lat': lat, 'Lon': lon, 
        'Altitude': name.astype('str'), 
        'Description':name.astype('str') 
       } 
df = pd.DataFrame(data=data) 
kml = simplekml.Kml() 
df.apply(lambda X: kml.newpoint(name=X["Name"], description=X["Description"], coords=[( X["Lon"],X["Lat"],X["Altitude"])]) ,axis=1) 
kml.save(path = "data.kml")